In [1]:
import uncertainpy as un
import numpy as np                   # For the time array
import chaospy as cp                 # To create distributions
from scipy.integrate import odeint   # To integrate our equation

In [2]:
import uncertainpy as un

import numpy as np
from scipy.integrate import odeint


# External stimulus
def I(time):
    return 140 # micro A/cm**2


def valderrama(V_0=-10,
               C_m=1,
               gbar_Na=120,
               gbar_K=36,
               gbar_L=0.3,
               E_Na=112,
               E_K=-12,
               E_l=10.613,
               m_0=0.0011,
               n_0=0.0003,
               h_0=0.9998):

    # Setup time
    end_time = 15          # ms
    dt = 0.025             # ms
    time = np.arange(0, end_time + dt, dt)

    # K channel
    def alpha_n(V):
        return 0.01*(10 - V)/(np.exp((10 - V)/10.) - 1)


    def beta_n(V):
        return 0.125*np.exp(-V/80.)

    def n_f(n, V):
        return alpha_n(V)*(1 - n) - beta_n(V)*n

    def n_inf(V):
        return alpha_n(V)/(alpha_n(V) + beta_n(V))


    # Na channel (activating)
    def alpha_m(V):
        return 0.1*(25 - V)/(np.exp((25 - V)/10.) - 1)

    def beta_m(V):
        return 4*np.exp(-V/18.)

    def m_f(m, V):
        return alpha_m(V)*(1 - m) - beta_m(V)*m

    def m_inf(V):
        return alpha_m(V)/(alpha_m(V) + beta_m(V))


    # Na channel (inactivating)
    def alpha_h(V):
        return 0.07*np.exp(-V/20.)

    def beta_h(V):
        return 1/(np.exp((30 - V)/10.) + 1)

    def h_f(h, V):
        return alpha_h(V)*(1 - h) - beta_h(V)*h

    def h_inf(V):
        return alpha_h(V)/(alpha_h(V) + beta_h(V))


    def dXdt(X, t):
        V, h, m, n = X

        g_Na = gbar_Na*(m**3)*h
        g_K = gbar_K*(n**4)
        g_l = gbar_L

        dmdt = m_f(m, V)
        dhdt = h_f(h, V)
        dndt = n_f(n, V)

        dVdt = (I(t) - g_Na*(V - E_Na) - g_K*(V - E_K) - g_l*(V - E_l))/C_m

        return [dVdt, dhdt, dmdt, dndt]


    initial_conditions = [V_0, h_0, m_0, n_0]

    X = odeint(dXdt, initial_conditions, time)
    values = X[:, 0]

    # Only return from 5 seconds onwards, as in the Valderrama paper
    values = values[time > 5]
    time = time[time > 5]

    # Add info needed by certain spiking features and efel features
    info = {"stimulus_start": time[0], "stimulus_end": time[-1]}

    return time, values, info

In [3]:
# Initialize the model
model = un.Model(run=valderrama,
                 labels=["Time (ms)", "Membrane potential (mV)"])

In [4]:
# Define a parameter dictionary
parameters = {"V_0": -10,
              "C_m": 1,
              "gbar_Na": 120,
              "gbar_K": 36,
              "gbar_L": 0.3,
              "m_0": 0.0011,
              "n_0": 0.0003,
              "h_0": 0.9998,
              "E_Na": 112,
              "E_K": -12,
              "E_l": 10.613
             }

# Create the parameters
parameters = un.Parameters(parameters)

In [5]:
# Set all parameters to have a uniform distribution
# within a 20% interval around their fixed value
parameters.set_all_distributions(un.uniform(0.2))

In [6]:
# Import the functions and libraries needed
#from uncertainpy.utils import create_logger
import logging

# Set up a logger. This adds a screen handlers to the "uncertainpy" logger
# if it does not already exist
# All log messages with level "info" or higher will be logged.
#setup_logger("uncertainpy.logger_name", level="info")

# Get the logger recently created
logger = logging.getLogger("uncertainpy.logger_name")
logger.setLevel(logging.INFO)

In [7]:
# Perform the uncertainty quantification
UQ = un.UncertaintyQuantification(model,
                                  parameters=parameters)

# We set the seed to easier be able to reproduce the result
data = UQ.quantify(seed=10,method="mc")#,nr_pc_mc_samples=100)

/Users/teo/.pyenv/versions/compstat/lib/python3.9/site-packages/SALib/sample/saltelli.py:94: UserWarning: 
        Convergence properties of the Sobol' sequence is only valid if
        `N` (5000) is equal to `2^n`.
        
  warnings.warn(msg)
Running model: 100%|█████████████████████| 65000/65000 [01:47<00:00, 604.75it/s]


Saving data as: data/valderrama.h5


https://github.com/simetenn/uncertainpy/issues/59

> I also had a similar experience. The "Calculating statistics from PCE" step takes aprox. 30s for 4 parameters, 3 minutes for 5 and around 1h for 6. I have not tried with more parameters. This step is not parallelized and I used a 3Ghz CPU. I wonder if this part of the code cannot be made more efficient as It seems to need a lot of time for > 6 parameters.



In [ ]:
# nest gets autistic